# Algorithm 18.1 (Local SQP Algorithm for solving (18.1))

In [ ]:
import numpy as np
from scipy.optimize import minimize

def objective(x):
    """ Define the objective function f_k. """
    return np.sum(x**2)  # Example: Quadratic function

def gradient(x):
    """ Compute the gradient \nabla f_k. """
    return 2 * x

def hessian(x):
    """ Compute the Hessian \nabla^2 L_k. """
    return 2 * np.eye(len(x))

def constraints(x):
    """ Define the constraint functions c_k. """
    return np.array([np.sum(x) - 1])  # Example: sum(x) = 1

def jacobian(x):
    """ Compute the Jacobian A_k. """
    return np.ones((1, len(x)))  # Derivative of sum(x)

def solve_qp(hess, grad, A, c):
    """ Solve the Quadratic Programming subproblem (18.7). """
    def qp_objective(p):
        return 0.5 * p @ hess @ p + grad @ p
    
    def qp_constraint(p):
        return A @ p + c
    
    constraints = { 'type': 'eq', 'fun': qp_constraint }
    result = minimize(qp_objective, np.zeros_like(grad), constraints=constraints)
    return result.x if result.success else None

def local_sqp(x0, lambda0, tol=1e-6, max_iter=100):
    x = x0
    lambda_ = lambda0
    k = 0
    
    while k < max_iter:
        f_k = objective(x)
        grad_f_k = gradient(x)
        hess_L_k = hessian(x)
        c_k = constraints(x)
        A_k = jacobian(x)
        
        p_k = solve_qp(hess_L_k, grad_f_k, A_k, c_k)
        if p_k is None or np.linalg.norm(p_k) < tol:
            break
        
        x = x + p_k
        lambda_ = -np.linalg.pinv(A_k @ A_k.T) @ A_k @ grad_f_k  # Lagrange multiplier update
        k += 1
    
    return x, lambda_

# Example usage
x0 = np.array([0.5, 0.5])  # Initial guess for x
lambda0 = np.array([0.0])  # Initial guess for lambda
solution, lambda_final = local_sqp(x0, lambda0)
print("Optimal x:", solution)
print("Optimal lambda:", lambda_final)


# Algorithm 16.3 (Line Search SQP Algorithm)

In [ ]:
def line_search_sqp(f, grad_f, c, A, x0, lambda0, eta=0.3, tau=0.5, tol=1e-6, max_iter=100):
    n = len(x0)
    m = len(lambda0)
    x_k = np.array(x0, dtype=float)
    lambda_k = np.array(lambda0, dtype=float)
    B_k = np.eye(n)  # Initial Hessian approximation if quasi-Newton is used
    
    def merit_function(x, mu):
        return f(x) + mu * np.linalg.norm(c(x), 1)
    
    for _ in range(max_iter):
        # Compute search direction p_k by solving the QP subproblem (18.11)
        grad_L = grad_f(x_k) - A(x_k).T @ lambda_k  # Gradient of Lagrangian
        p_k = -np.linalg.solve(B_k, grad_L)  # Newton step (approximation)
        lambda_hat = np.linalg.solve(A(x_k) @ A(x_k).T, A(x_k) @ grad_f(x_k))
        p_lambda = lambda_hat - lambda_k
        
        # Line search
        mu_k = 1  # Initial choice
        alpha_k = 1
        while merit_function(x_k + alpha_k * p_k, mu_k) > \
              merit_function(x_k, mu_k) + eta * alpha_k * np.dot(grad_L, p_k):
            alpha_k *= tau  # Reduce step size
        
        # Update iterates
        x_k += alpha_k * p_k
        lambda_k += alpha_k * p_lambda
        
        # Convergence check
        if np.linalg.norm(p_k) < tol:
            break
        
        # Quasi-Newton update
        s_k = alpha_k * p_k
        y_k = grad_f(x_k) - grad_f(x_k - s_k)
        if np.dot(s_k, y_k) > 0:
            B_k += np.outer(y_k, y_k) / np.dot(y_k, s_k) - np.outer(B_k @ s_k, B_k @ s_k) / np.dot(s_k, B_k @ s_k)
    
    return x_k, lambda_k


# Algoritma 16.4 (Byrd-Omojokun Trust-Region SQP Method) 

In [6]:
def compute_f(x):
    # Define the objective function f(x)
    pass

def compute_c(x):
    # Define the constraint function c(x)
    pass

def compute_grad_f(x):
    # Compute the gradient of f
    pass

def compute_A(x):
    # Compute the Jacobian of the constraints
    pass

def compute_multiplier_estimates(A, grad_f):
    # Compute Lagrange multiplier estimates
    return np.linalg.solve(A @ A.T, A @ grad_f)

def solve_subproblem(v_k, r_k):
    # Solve the subproblem to get v_k and compute r_k
    pass

def compute_hessian_or_quasi_newton(x):
    # Compute Hessian \nabla^2_x L or a quasi-Newton approximation
    pass

def projected_CG_method(H, grad_f, A):
    # Compute search direction p_k using Projected Conjugate Gradient method
    pass

def choose_mu():
    # Choose regularization parameter \mu_k
    pass

def compute_ratio(ared, pred):
    # Compute rho_k = ared_k / pred_k
    return ared / pred if pred != 0 else 0

def byrd_omojokun_trust_region_sqp(x0, delta0, epsilon, gamma, eta, max_iters=100):
    x_k = x0
    delta_k = delta0
    
    for k in range(max_iters):
        f_k = compute_f(x_k)
        c_k = compute_c(x_k)
        grad_f_k = compute_grad_f(x_k)
        A_k = compute_A(x_k)
        
        lambda_k = compute_multiplier_estimates(A_k, grad_f_k)
        
        if np.linalg.norm(grad_f_k - A_k.T @ lambda_k, np.inf) < epsilon and np.linalg.norm(c_k, np.inf) < epsilon:
            return x_k  # Stop with approximate solution
        
        v_k, r_k = solve_subproblem(None, None)  # Placeholder
        H_k = compute_hessian_or_quasi_newton(x_k)
        p_k = projected_CG_method(H_k, grad_f_k, A_k)
        mu_k = choose_mu()
        
        # Compute actual and predicted reduction (placeholders for now)
        ared_k, pred_k = 1, 1  # Replace with proper computations
        rho_k = compute_ratio(ared_k, pred_k)
        
        if rho_k > eta:
            x_k = x_k + p_k
            delta_k = max(delta_k, delta_k)  # Ensure Delta_{k+1} >= Delta_k
        else:
            delta_k = min(delta_k, gamma * np.linalg.norm(p_k))
        
    return x_k


# Algorithm 18.5 (Penalty Update and Step Computation)

In [ ]:
def penalty_update(x_k, mu_k_minus_1, Delta_k, epsilon_1, epsilon_2, m_k, p, q):
    """
    Implements Algorithm 18.5 (Penalty Update and Step Computation).
    
    Parameters:
    x_k : current iterate (not used in this function but typically relevant in broader optimization context)
    mu_k_minus_1 : previous penalty parameter
    Delta_k : trust-region radius (not directly used here)
    epsilon_1, epsilon_2 : algorithm parameters (0,1)
    m_k : function m_k(y) (model function at iteration k)
    p : function p(mu) solving the subproblem
    q : function q(y) (another function required for the final condition)
    
    Returns:
    mu_k : updated penalty parameter
    p_k : updated step computation
    """
    # Solve subproblem to get p(mu_k_minus_1)
    p_mu_k_minus_1 = p(mu_k_minus_1)
    
    if m_k(p_mu_k_minus_1) == 0:
        mu_plus = mu_k_minus_1
    else:
        # Compute p_infinity
        p_infinity = p(float('inf'))  # Assuming p(∞) can be approximated numerically
        
        if m_k(p_infinity) == 0:
            # Find mu^+ > mu_k_minus_1 such that m_k(p(mu^+)) = 0
            mu_plus = mu_k_minus_1 * 1.1  # Increase mu slightly (heuristic choice)
            while m_k(p(mu_plus)) != 0:
                mu_plus *= 1.1  # Increase mu until condition is met
        else:
            # Find mu^+ ≥ mu_k_minus_1 such that m_k(0) - m_k(p(mu^+)) ≥ ε₁ [m_k(0) - m_k(p_infinity)]
            mu_plus = mu_k_minus_1 * 1.1
            while m_k(0) - m_k(p(mu_plus)) < epsilon_1 * (m_k(0) - m_k(p_infinity)):
                mu_plus *= 1.1
    
    # Increase mu^+ if necessary to satisfy q condition
    while q(0) - q(p(mu_plus)) < epsilon_2 * mu_plus * (m_k(0) - m_k(p(mu_plus))):
        mu_plus *= 1.1
    
    # Update values
    mu_k = mu_plus
    p_k = p(mu_plus)
    
    return mu_k, p_k
